In [3]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import random
from bs4 import BeautifulSoup
import requests

In [2]:
pip install pymysql

   ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
   --------- ------------------------------ 10.2/44.8 kB ? eta -:--:--
   --------- ------------------------------ 10.2/44.8 kB ? eta -:--:--
   --------- ------------------------------ 10.2/44.8 kB ? eta -:--:--
   --------- ------------------------------ 10.2/44.8 kB ? eta -:--:--
   --------- ------------------------------ 10.2/44.8 kB ? eta -:--:--
   ------------------ --------------------- 20.5/44.8 kB 46.8 kB/s eta 0:00:01
   ------------------ --------------------- 20.5/44.8 kB 46.8 kB/s eta 0:00:01
   ------------------ --------------------- 20.5/44.8 kB 46.8 kB/s eta 0:00:01
   ------------------ --------------------- 20.5/44.8 kB 4

In [4]:
# Créer une connexion à MySQL sans spécifier de base de données

engine = create_engine('mysql+pymysql://root:@localhost/aistoredb')

# Categorie


In [5]:
df_cat = pd.read_csv('category.csv', encoding='latin-1')
data_cat = df_cat.copy()

In [6]:
data_cat

,Type
0,Charcuterie
1,Condiment
2,Produits de rasage
3,Produits de soin pour les ongles
4,Thé
5,Produits d'hygiène
6,Produits de soin pour le visage
7,Liqueur
8,Alimentation
9,Hygiène


In [7]:
# Réinitialiser l'index du DataFrame pour qu'il commence à partir de 1
data_cat.reset_index(drop=True, inplace=True)
data_cat.index += 1


In [8]:
# Créer une nouvelle colonne 'id_category' en utilisant les index du DataFrame
data_cat['id_category'] = data_cat.index

# Renommer la colonne 'Name' en 'nom_category'
data_cat.rename(columns={'Type': 'nom_category'}, inplace=True)

In [9]:
data_cat

,nom_category,id_category
1,Charcuterie,1
2,Condiment,2
3,Produits de rasage,3
4,Produits de soin pour les ongles,4
5,Thé,5
6,Produits d'hygiène,6
7,Produits de soin pour le visage,7
8,Liqueur,8
9,Alimentation,9
10,Hygiène,10


In [10]:
data_cat[['id_category', 'nom_category']].to_sql(name='stock_category', if_exists='replace', con=engine, index=False)

40

# Produits

In [11]:
df_prod = pd.read_csv('produits.csv', encoding='latin-1', sep=';')
data_prod = df_prod.copy()

In [12]:
data_prod

,Produit,Category,Prix
0,Saucisson à l'anis,Charcuterie,1400
1,Pâté de lapin aux noix,Charcuterie,1800
2,Sachet de sel,Condiment,500
3,Terrine de faisan aux morilles,Charcuterie,1700
4,Baume après-rasage hydratant Nivea,Produits de rasage,700
...,...,...,...
317,Sachet de thé rooibos,Thé,500
318,Lotion anti-imperfections Garnier,Produits de soin pour le visage,1200
319,Déodorant bille sans parfum Vichy,Produits d'hygiène,800
320,Soin contour des yeux L'Oréal,Produits de soin pour le visage,2500


In [13]:
# Effectuer une jointure entre data_prod et data_cat pour obtenir les identifiants correspondants

data_prod = data_prod.merge(data_cat, left_on='Category', right_on='nom_category', how='left')

In [14]:
data_prod

,Produit,Category,Prix,nom_category,id_category
0,Saucisson à l'anis,Charcuterie,1400,Charcuterie,1
1,Pâté de lapin aux noix,Charcuterie,1800,Charcuterie,1
2,Sachet de sel,Condiment,500,Condiment,2
3,Terrine de faisan aux morilles,Charcuterie,1700,Charcuterie,1
4,Baume après-rasage hydratant Nivea,Produits de rasage,700,Produits de rasage,3
...,...,...,...,...,...
317,Sachet de thé rooibos,Thé,500,Thé,5
318,Lotion anti-imperfections Garnier,Produits de soin pour le visage,1200,Produits de soin pour le visage,7
319,Déodorant bille sans parfum Vichy,Produits d'hygiène,800,Produits d'hygiène,6
320,Soin contour des yeux L'Oréal,Produits de soin pour le visage,2500,Produits de soin pour le visage,7


In [15]:
# Supprimer les colonnes non nécessaire

data_prod.drop(columns=['Category', 'nom_category'], inplace=True, axis=1)

In [16]:
data_prod

,Produit,Prix,id_category
0,Saucisson à l'anis,1400,1
1,Pâté de lapin aux noix,1800,1
2,Sachet de sel,500,2
3,Terrine de faisan aux morilles,1700,1
4,Baume après-rasage hydratant Nivea,700,3
...,...,...,...
317,Sachet de thé rooibos,500,5
318,Lotion anti-imperfections Garnier,1200,7
319,Déodorant bille sans parfum Vichy,800,6
320,Soin contour des yeux L'Oréal,2500,7


In [17]:
# Réinitialiser l'index du DataFrame pour qu'il commence à partir de 1
data_prod.reset_index(drop=True, inplace=True)
data_prod.index += 1


# Créer une nouvelle colonne 'id_category' en utilisant les index du DataFrame

data_prod['id'] = data_prod.index

In [18]:
data_prod

,Produit,Prix,id_category,id
1,Saucisson à l'anis,1400,1,1
2,Pâté de lapin aux noix,1800,1,2
3,Sachet de sel,500,2,3
4,Terrine de faisan aux morilles,1700,1,4
5,Baume après-rasage hydratant Nivea,700,3,5
...,...,...,...,...
318,Sachet de thé rooibos,500,5,318
319,Lotion anti-imperfections Garnier,1200,7,319
320,Déodorant bille sans parfum Vichy,800,6,320
321,Soin contour des yeux L'Oréal,2500,7,321


In [19]:
# Assigner des valeurs à plusieurs colonnes simultanément

data_prod['quantite'] = [0] * len(data_prod)  # Assigner 0 à toutes les lignes de la colonne 'quantite'
data_prod['is_deleted'] = [0] * len(data_prod)  # Assigner 0 à toutes les lignes de la colonne 'is_deleted'

In [20]:
data_prod

,Produit,Prix,id_category,id,quantite,is_deleted
1,Saucisson à l'anis,1400,1,1,0,0
2,Pâté de lapin aux noix,1800,1,2,0,0
3,Sachet de sel,500,2,3,0,0
4,Terrine de faisan aux morilles,1700,1,4,0,0
5,Baume après-rasage hydratant Nivea,700,3,5,0,0
...,...,...,...,...,...,...
318,Sachet de thé rooibos,500,5,318,0,0
319,Lotion anti-imperfections Garnier,1200,7,319,0,0
320,Déodorant bille sans parfum Vichy,800,6,320,0,0
321,Soin contour des yeux L'Oréal,2500,7,321,0,0


In [21]:
# Renommer les colonnes 'produit' en 'nom' et 'id_category' en 'id_category_id'

data_prod.rename(columns={'Produit': 'nom', 'id_category': 'id_category_id', 'Prix': 'prix'}, inplace=True)
data_prod

,nom,prix,id_category_id,id,quantite,is_deleted
1,Saucisson à l'anis,1400,1,1,0,0
2,Pâté de lapin aux noix,1800,1,2,0,0
3,Sachet de sel,500,2,3,0,0
4,Terrine de faisan aux morilles,1700,1,4,0,0
5,Baume après-rasage hydratant Nivea,700,3,5,0,0
...,...,...,...,...,...,...
318,Sachet de thé rooibos,500,5,318,0,0
319,Lotion anti-imperfections Garnier,1200,7,319,0,0
320,Déodorant bille sans parfum Vichy,800,6,320,0,0
321,Soin contour des yeux L'Oréal,2500,7,321,0,0


In [22]:
data_prod[['id', 'nom', 'quantite', 'is_deleted', 'id_category_id', 'prix']].to_sql(name='stock_stock', if_exists='replace', con=engine, index=False)

322

# Ventes


In [23]:
df_ven = pd.read_csv('Fichier_des_ventes.csv', delimiter=';')
data_ven = df_ven.copy()
data_ven

,id,Name,Gender,Produit,Type,Prix,Date
0,1,Mark Green,Male,Saucisson à l'anis,Charcuterie,1400,2022-11-18 02:48:00.0000000
1,2,Chukwudi Eze,Male,Pâté de lapin aux noix,Charcuterie,1800,2022-04-26 20:34:00.0000000
2,3,Babatunde Adewale,Male,Sachet de sel,Condiment,500,2022-11-14 13:08:00.0000000
3,4,Victoria Young,Female,Terrine de faisan aux morilles,Charcuterie,1700,2022-07-23 09:55:00.0000000
4,5,Fanta Camara,Female,Baume après-rasage hydratant Nivea,Produits de rasage,700,2022-11-19 13:01:00.0000000
...,...,...,...,...,...,...,...
2578,2579,Sebastian Wright,Male,Rillettes de dinde,Charcuterie,1300,2022-11-09 12:22:00.0000000
2579,2580,Olivia Smith,Female,Pot de crème dessert Danette,Dessert,1000,2022-11-01 10:35:00.0000000
2580,2581,Henry Perry,Male,Baume pour les lèvres Carmex,Produits de soin des lèvres,700,2022-11-18 11:44:00.0000000
2581,2582,Christine Young,Female,Pâté de lièvre,Charcuterie,1800,2022-11-04 23:53:00.0000000


In [24]:
# Effectuer une jointure entre products_df et categories_df pour obtenir les identifiants correspondants

data_ven = data_ven.merge(data_prod, left_on='Produit', right_on='nom', how='left')
data_ven

,id_x,Name,Gender,Produit,Type,Prix,Date,nom,prix,id_category_id,id_y,quantite,is_deleted
0,1,Mark Green,Male,Saucisson à l'anis,Charcuterie,1400,2022-11-18 02:48:00.0000000,Saucisson à l'anis,1400,1,1,0,0
1,2,Chukwudi Eze,Male,Pâté de lapin aux noix,Charcuterie,1800,2022-04-26 20:34:00.0000000,Pâté de lapin aux noix,1800,1,2,0,0
2,3,Babatunde Adewale,Male,Sachet de sel,Condiment,500,2022-11-14 13:08:00.0000000,Sachet de sel,500,2,3,0,0
3,4,Victoria Young,Female,Terrine de faisan aux morilles,Charcuterie,1700,2022-07-23 09:55:00.0000000,Terrine de faisan aux morilles,1700,1,4,0,0
4,5,Fanta Camara,Female,Baume après-rasage hydratant Nivea,Produits de rasage,700,2022-11-19 13:01:00.0000000,Baume après-rasage hydratant Nivea,700,3,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2578,2579,Sebastian Wright,Male,Rillettes de dinde,Charcuterie,1300,2022-11-09 12:22:00.0000000,Rillettes de dinde,1300,1,169,0,0
2579,2580,Olivia Smith,Female,Pot de crème dessert Danette,Dessert,1000,2022-11-01 10:35:00.0000000,Pot de crème dessert Danette,1000,34,243,0,0
2580,2581,Henry Perry,Male,Baume pour les lèvres Carmex,Produits de soin des lèvres,700,2022-11-18 11:44:00.0000000,Baume pour les lèvres Carmex,700,21,222,0,0
2581,2582,Christine Young,Female,Pâté de lièvre,Charcuterie,1800,2022-11-04 23:53:00.0000000,Pâté de lièvre,1800,1,290,0,0


In [25]:
# Supprimer les colonnes non nécessaire

data_ven.drop(columns=['Produit', 'is_deleted', 'id_category_id', 'nom', 'Prix', 'Type'], inplace=True, axis=1)
data_ven

,id_x,Name,Gender,Date,prix,id_y,quantite
0,1,Mark Green,Male,2022-11-18 02:48:00.0000000,1400,1,0
1,2,Chukwudi Eze,Male,2022-04-26 20:34:00.0000000,1800,2,0
2,3,Babatunde Adewale,Male,2022-11-14 13:08:00.0000000,500,3,0
3,4,Victoria Young,Female,2022-07-23 09:55:00.0000000,1700,4,0
4,5,Fanta Camara,Female,2022-11-19 13:01:00.0000000,700,5,0
...,...,...,...,...,...,...,...
2578,2579,Sebastian Wright,Male,2022-11-09 12:22:00.0000000,1300,169,0
2579,2580,Olivia Smith,Female,2022-11-01 10:35:00.0000000,1000,243,0
2580,2581,Henry Perry,Male,2022-11-18 11:44:00.0000000,700,222,0
2581,2582,Christine Young,Female,2022-11-04 23:53:00.0000000,1800,290,0


In [26]:
# Renommer les colonnes 

data_ven.rename(columns={'id_x': 'id','Name': 'nom', 'prix': 'prixunitaire', 'Gender': 'genre', 'id_y': 'stock_id', 'Date': 'date'}, inplace=True)
data_ven

,id,nom,genre,date,prixunitaire,stock_id,quantite
0,1,Mark Green,Male,2022-11-18 02:48:00.0000000,1400,1,0
1,2,Chukwudi Eze,Male,2022-04-26 20:34:00.0000000,1800,2,0
2,3,Babatunde Adewale,Male,2022-11-14 13:08:00.0000000,500,3,0
3,4,Victoria Young,Female,2022-07-23 09:55:00.0000000,1700,4,0
4,5,Fanta Camara,Female,2022-11-19 13:01:00.0000000,700,5,0
...,...,...,...,...,...,...,...
2578,2579,Sebastian Wright,Male,2022-11-09 12:22:00.0000000,1300,169,0
2579,2580,Olivia Smith,Female,2022-11-01 10:35:00.0000000,1000,243,0
2580,2581,Henry Perry,Male,2022-11-18 11:44:00.0000000,700,222,0
2581,2582,Christine Young,Female,2022-11-04 23:53:00.0000000,1800,290,0


In [27]:
# Réinitialiser l'index du DataFrame pour qu'il commence à partir de 1
data_ven.reset_index(drop=True, inplace=True)
data_ven.index += 1


# Créer une nouvelle colonne 'nofacture_id' en utilisant les index du DataFrame

data_ven['nofacture_id'] = data_ven.index
data_ven

,id,nom,genre,date,prixunitaire,stock_id,quantite,nofacture_id
1,1,Mark Green,Male,2022-11-18 02:48:00.0000000,1400,1,0,1
2,2,Chukwudi Eze,Male,2022-04-26 20:34:00.0000000,1800,2,0,2
3,3,Babatunde Adewale,Male,2022-11-14 13:08:00.0000000,500,3,0,3
4,4,Victoria Young,Female,2022-07-23 09:55:00.0000000,1700,4,0,4
5,5,Fanta Camara,Female,2022-11-19 13:01:00.0000000,700,5,0,5
...,...,...,...,...,...,...,...,...
2579,2579,Sebastian Wright,Male,2022-11-09 12:22:00.0000000,1300,169,0,2579
2580,2580,Olivia Smith,Female,2022-11-01 10:35:00.0000000,1000,243,0,2580
2581,2581,Henry Perry,Male,2022-11-18 11:44:00.0000000,700,222,0,2581
2582,2582,Christine Young,Female,2022-11-04 23:53:00.0000000,1800,290,0,2582


In [28]:
# Assigner des valeurs à la colonnes quantité

data_ven['quantite'] = [1] * len(data_ven)  # Assigner 1 à toutes les lignes de la colonne 'quantite'
data_ven

,id,nom,genre,date,prixunitaire,stock_id,quantite,nofacture_id
1,1,Mark Green,Male,2022-11-18 02:48:00.0000000,1400,1,1,1
2,2,Chukwudi Eze,Male,2022-04-26 20:34:00.0000000,1800,2,1,2
3,3,Babatunde Adewale,Male,2022-11-14 13:08:00.0000000,500,3,1,3
4,4,Victoria Young,Female,2022-07-23 09:55:00.0000000,1700,4,1,4
5,5,Fanta Camara,Female,2022-11-19 13:01:00.0000000,700,5,1,5
...,...,...,...,...,...,...,...,...
2579,2579,Sebastian Wright,Male,2022-11-09 12:22:00.0000000,1300,169,1,2579
2580,2580,Olivia Smith,Female,2022-11-01 10:35:00.0000000,1000,243,1,2580
2581,2581,Henry Perry,Male,2022-11-18 11:44:00.0000000,700,222,1,2581
2582,2582,Christine Young,Female,2022-11-04 23:53:00.0000000,1800,290,1,2582


In [29]:
# Créer la colonne "montant"

data_ven['montant'] = data_ven['prixunitaire'] * data_ven['quantite']
data_ven

,id,nom,genre,date,prixunitaire,stock_id,quantite,nofacture_id,montant
1,1,Mark Green,Male,2022-11-18 02:48:00.0000000,1400,1,1,1,1400
2,2,Chukwudi Eze,Male,2022-04-26 20:34:00.0000000,1800,2,1,2,1800
3,3,Babatunde Adewale,Male,2022-11-14 13:08:00.0000000,500,3,1,3,500
4,4,Victoria Young,Female,2022-07-23 09:55:00.0000000,1700,4,1,4,1700
5,5,Fanta Camara,Female,2022-11-19 13:01:00.0000000,700,5,1,5,700
...,...,...,...,...,...,...,...,...,...
2579,2579,Sebastian Wright,Male,2022-11-09 12:22:00.0000000,1300,169,1,2579,1300
2580,2580,Olivia Smith,Female,2022-11-01 10:35:00.0000000,1000,243,1,2580,1000
2581,2581,Henry Perry,Male,2022-11-18 11:44:00.0000000,700,222,1,2581,700
2582,2582,Christine Young,Female,2022-11-04 23:53:00.0000000,1800,290,1,2582,1800


In [30]:
# Liste des préfixes de téléphone valides
prefixes = ['01', '05', '07']

# Générer aléatoirement des numéros de téléphone
telephones = [random.choice(prefixes) + ''.join(random.choices('0123456789', k=8)) for _ in range(len(data_ven))]

# Créer une nouvelle colonne "telephone" et lui assigner les valeurs générées aléatoirement
data_ven = data_ven.assign(telephone=telephones)
data_ven

,id,nom,genre,date,prixunitaire,stock_id,quantite,nofacture_id,montant,telephone
1,1,Mark Green,Male,2022-11-18 02:48:00.0000000,1400,1,1,1,1400,0719012976
2,2,Chukwudi Eze,Male,2022-04-26 20:34:00.0000000,1800,2,1,2,1800,0523103616
3,3,Babatunde Adewale,Male,2022-11-14 13:08:00.0000000,500,3,1,3,500,0574313930
4,4,Victoria Young,Female,2022-07-23 09:55:00.0000000,1700,4,1,4,1700,0763526842
5,5,Fanta Camara,Female,2022-11-19 13:01:00.0000000,700,5,1,5,700,0527553223
...,...,...,...,...,...,...,...,...,...,...
2579,2579,Sebastian Wright,Male,2022-11-09 12:22:00.0000000,1300,169,1,2579,1300,0573277864
2580,2580,Olivia Smith,Female,2022-11-01 10:35:00.0000000,1000,243,1,2580,1000,0583120042
2581,2581,Henry Perry,Male,2022-11-18 11:44:00.0000000,700,222,1,2581,700,0768572438
2582,2582,Christine Young,Female,2022-11-04 23:53:00.0000000,1800,290,1,2582,1800,0150035193


In [31]:
# Dictionnaire des rues par commune
rues_par_commune = {
    'Cocody': ['Rue des Jardins', 'Avenue des Cocotiers', 'Boulevard des Martyrs', 'Cité SIRS', 'Cité des arts', 'Cité rouge', 'Cité mermose', 'Riveira Golf', 'Riviéra Palmerai', 'Riviéra Bounoumin', 'Riviéra Attoban', 'Riviéra Faya'],
    'Treichville': ['Rue des Banques', 'Avenue 18', 'Boulevard Djé Bré', 'Rue 12 Avenue 21'],
    'Plateau': ['Rue Léon Blum', 'Avenue Chardy', 'Boulevard de la République'],
    'Adjamé': ['220 Logements', 'Liberté', 'Agban Gendarmerie', 'Chicane', 'CRS', 'Agban Village', 'Bromacoté', 'Dallas', 'Paillet', 'Williamsville'],
    'Abobo': ['Avocatier', 'Kenedy', 'Sogefia', 'Banco', 'Anador', 'Dokui', 'Azur', 'Biabou', 'PK18', "N'dotre"],
    'Yopougon': ['Bel Air', 'Selmer', 'Andokoi', 'Maroc', 'Gesco', 'Ananeraie', 'kenya', 'Niangon'],
    'Bingerville': ['Bandji', 'Feh Kessé', 'Gbagba'],
    'Anyama': ['Ancien gendarmerie', 'Dar Salam', 'Schneider', 'Ebimpe'],
    'Koumassi': ['3 Ampoules', 'Divo', 'Soweto'],
    'Marcory': ['Zone 4', 'Residentiel', 'Bietry', 'INJS']
}

# Générer aléatoirement des adresses en fonction de la commune
def generer_adresse():
    # Choisir aléatoirement une commune parmi celles disponibles dans le dictionnaire
    commune = random.choice(list(rues_par_commune.keys()))
    # Choisir aléatoirement une rue parmi celles disponibles pour cette commune
    rue = random.choice(rues_par_commune[commune])
    # Retourner l'adresse avec la commune et Abidjan
    return f"{rue}, {commune}, Abidjan"

# Créer une nouvelle colonne "adresse" en utilisant la fonction generer_adresse
data_ven['adresse'] = data_ven.apply(lambda row: generer_adresse(), axis=1)
data_ven

,id,nom,genre,date,prixunitaire,stock_id,quantite,nofacture_id,montant,telephone,adresse
1,1,Mark Green,Male,2022-11-18 02:48:00.0000000,1400,1,1,1,1400,0719012976,"Cité rouge, Cocody, Abidjan"
2,2,Chukwudi Eze,Male,2022-04-26 20:34:00.0000000,1800,2,1,2,1800,0523103616,"Rue des Banques, Treichville, Abidjan"
3,3,Babatunde Adewale,Male,2022-11-14 13:08:00.0000000,500,3,1,3,500,0574313930,"Biabou, Abobo, Abidjan"
4,4,Victoria Young,Female,2022-07-23 09:55:00.0000000,1700,4,1,4,1700,0763526842,"Cité rouge, Cocody, Abidjan"
5,5,Fanta Camara,Female,2022-11-19 13:01:00.0000000,700,5,1,5,700,0527553223,"Dallas, Adjamé, Abidjan"
...,...,...,...,...,...,...,...,...,...,...,...
2579,2579,Sebastian Wright,Male,2022-11-09 12:22:00.0000000,1300,169,1,2579,1300,0573277864,"Maroc, Yopougon, Abidjan"
2580,2580,Olivia Smith,Female,2022-11-01 10:35:00.0000000,1000,243,1,2580,1000,0583120042,"Residentiel, Marcory, Abidjan"
2581,2581,Henry Perry,Male,2022-11-18 11:44:00.0000000,700,222,1,2581,700,0768572438,"Rue des Banques, Treichville, Abidjan"
2582,2582,Christine Young,Female,2022-11-04 23:53:00.0000000,1800,290,1,2582,1800,0150035193,"Boulevard Djé Bré, Treichville, Abidjan"


In [32]:
# Liste des domaines d'e-mails possibles
domaines_emails = ['gmail.com', 'gmail.com', 'gmail.com', 'gmail.com', 'yahoo.com', 'hotmail.com']

# Liste des différentes structures d'e-mails possibles
structures_emails = ['prenom.nom', 'prenom_nom', 'nom.prenom', 'nom_prenom', 'nomprenom']

# Fonction pour générer un nombre aléatoire à ajouter à l'email
def generer_nombre_aleatoire():
    return random.randint(100, 999)

# Fonction pour générer l'email à partir du nom et du prénom
def generer_email(nom):
    # Séparer le nom et le prénom
    nom_prenom = nom.split()
    prenom = nom_prenom[0].lower()  # Utiliser le prénom en minuscules
    nom = nom_prenom[-1].lower()     # Utiliser le nom en minuscules
    # Générer un nombre aléatoire
    nombre_aleatoire = generer_nombre_aleatoire()
    # Choisir aléatoirement une structure d'e-mail parmi la liste des structures possibles
    structure_email = random.choice(structures_emails)
    # Choisir un domaine d'email aléatoire parmi la liste des domaines possibles
    domaine_email = random.choice(domaines_emails)
    # Concaténer les parties de l'e-mail en fonction de la structure choisie
    if structure_email == 'prenom.nom':
        return f"{prenom}.{nom}{nombre_aleatoire}@{domaine_email}"
    elif structure_email == 'prenom_nom':
        return f"{prenom}_{nom}{nombre_aleatoire}@{domaine_email}"
    elif structure_email == 'nom.prenom':
        return f"{nom}.{prenom}{nombre_aleatoire}@{domaine_email}"
    elif structure_email == 'nom_prenom':
        return f"{nom}_{prenom}{nombre_aleatoire}@{domaine_email}"
    elif structure_email == 'nomprenom':
        return f"{nom}{prenom}{nombre_aleatoire}@{domaine_email}"
    else:
        return f"{prenom}{nom}{nombre_aleatoire}@{domaine_email}"  # Par défaut, utiliser la structure prenom.nom

# Créer une nouvelle colonne "email" en utilisant la fonction generer_email
data_ven['email'] = data_ven['nom'].apply(generer_email)
data_ven

,id,nom,genre,date,prixunitaire,stock_id,quantite,nofacture_id,montant,telephone,adresse,email
1,1,Mark Green,Male,2022-11-18 02:48:00.0000000,1400,1,1,1,1400,0719012976,"Cité rouge, Cocody, Abidjan",green.mark886@gmail.com
2,2,Chukwudi Eze,Male,2022-04-26 20:34:00.0000000,1800,2,1,2,1800,0523103616,"Rue des Banques, Treichville, Abidjan",chukwudi.eze506@gmail.com
3,3,Babatunde Adewale,Male,2022-11-14 13:08:00.0000000,500,3,1,3,500,0574313930,"Biabou, Abobo, Abidjan",adewalebabatunde175@gmail.com
4,4,Victoria Young,Female,2022-07-23 09:55:00.0000000,1700,4,1,4,1700,0763526842,"Cité rouge, Cocody, Abidjan",young.victoria532@yahoo.com
5,5,Fanta Camara,Female,2022-11-19 13:01:00.0000000,700,5,1,5,700,0527553223,"Dallas, Adjamé, Abidjan",fanta.camara983@hotmail.com
...,...,...,...,...,...,...,...,...,...,...,...,...
2579,2579,Sebastian Wright,Male,2022-11-09 12:22:00.0000000,1300,169,1,2579,1300,0573277864,"Maroc, Yopougon, Abidjan",sebastian.wright628@gmail.com
2580,2580,Olivia Smith,Female,2022-11-01 10:35:00.0000000,1000,243,1,2580,1000,0583120042,"Residentiel, Marcory, Abidjan",smith.olivia773@gmail.com
2581,2581,Henry Perry,Male,2022-11-18 11:44:00.0000000,700,222,1,2581,700,0768572438,"Rue des Banques, Treichville, Abidjan",henry_perry633@gmail.com
2582,2582,Christine Young,Female,2022-11-04 23:53:00.0000000,1800,290,1,2582,1800,0150035193,"Boulevard Djé Bré, Treichville, Abidjan",youngchristine108@gmail.com


In [33]:
data_fact = data_ven[['id', 'date', 'nom', 'telephone', 'adresse', 'email', 'genre']]
data_fact

,id,date,nom,telephone,adresse,email,genre
1,1,2022-11-18 02:48:00.0000000,Mark Green,0719012976,"Cité rouge, Cocody, Abidjan",green.mark886@gmail.com,Male
2,2,2022-04-26 20:34:00.0000000,Chukwudi Eze,0523103616,"Rue des Banques, Treichville, Abidjan",chukwudi.eze506@gmail.com,Male
3,3,2022-11-14 13:08:00.0000000,Babatunde Adewale,0574313930,"Biabou, Abobo, Abidjan",adewalebabatunde175@gmail.com,Male
4,4,2022-07-23 09:55:00.0000000,Victoria Young,0763526842,"Cité rouge, Cocody, Abidjan",young.victoria532@yahoo.com,Female
5,5,2022-11-19 13:01:00.0000000,Fanta Camara,0527553223,"Dallas, Adjamé, Abidjan",fanta.camara983@hotmail.com,Female
...,...,...,...,...,...,...,...
2579,2579,2022-11-09 12:22:00.0000000,Sebastian Wright,0573277864,"Maroc, Yopougon, Abidjan",sebastian.wright628@gmail.com,Male
2580,2580,2022-11-01 10:35:00.0000000,Olivia Smith,0583120042,"Residentiel, Marcory, Abidjan",smith.olivia773@gmail.com,Female
2581,2581,2022-11-18 11:44:00.0000000,Henry Perry,0768572438,"Rue des Banques, Treichville, Abidjan",henry_perry633@gmail.com,Male
2582,2582,2022-11-04 23:53:00.0000000,Christine Young,0150035193,"Boulevard Djé Bré, Treichville, Abidjan",youngchristine108@gmail.com,Female


In [34]:
data_fact.rename(columns={'id': 'nofacture'}, inplace=True)
data_fact[['nofacture', 'date', 'nom', 'telephone', 'adresse', 'email', 'genre']].to_sql(name='transactions_facturevente', if_exists='append', con=engine, index=False)

C:\Users\mom_by\AppData\Local\Temp\ipykernel_16092\2418576939.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fact.rename(columns={'id': 'nofacture'}, inplace=True)


2583

In [35]:


data_art = data_ven[['id', 'quantite', 'montant', 'nofacture_id', 'stock_id']]
data_art

,id,quantite,montant,nofacture_id,stock_id
1,1,1,1400,1,1
2,2,1,1800,2,2
3,3,1,500,3,3
4,4,1,1700,4,4
5,5,1,700,5,5
...,...,...,...,...,...
2579,2579,1,1300,2579,169
2580,2580,1,1000,2580,243
2581,2581,1,700,2581,222
2582,2582,1,1800,2582,290


In [36]:
data_art.to_sql(name='transactions_articlevente', if_exists='append', con=engine, index=False)

2583

In [37]:
data_det = data_ven[['id', 'montant', 'stock_id']]
data_det.rename(columns={'montant': 'total', 'stock_id': 'nofacture_id'}, inplace=True)
data_det

C:\Users\mom_by\AppData\Local\Temp\ipykernel_16092\3475504031.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_det.rename(columns={'montant': 'total', 'stock_id': 'nofacture_id'}, inplace=True)


,id,total,nofacture_id
1,1,1400,1
2,2,1800,2
3,3,500,3
4,4,1700,4
5,5,700,5
...,...,...,...
2579,2579,1300,169
2580,2580,1000,243
2581,2581,700,222
2582,2582,1800,290


In [38]:
data_det.to_sql(name='transactions_detailsfacturevente', if_exists='append', con=engine, index=False)

2583

# fournisseur 

In [39]:
# URL de le site
url = 'https://www.goafricaonline.com/ci/annuaire/distributeur-grossiste'

In [40]:
# Envoyer une requête GET à l'URL
response = requests.get(url)

# Analyser le contenu HTML de la page
soup = BeautifulSoup(response.text, 'html.parser')

In [41]:
# Trouver le tableau de la liste des pays
table = soup.find('article')

In [42]:
# Créer une liste vide pour stocker les données
data = []

In [43]:
# Fonction pour extraire les informations d'un article
def extract_article_info(article):
    nom = article.find("h2").text.strip()
    adresse = article.find("address").text.strip().replace("<br/>", ", ")
    telephone = article.find("a", href=lambda href: href and "tel" in href).text.strip()
    email = article.find("a", href=lambda href: href and "@" in href).text.strip()
    return {"nom": nom, "adresse": adresse, "telephone": telephone, "email": email}

In [44]:
def extract_article_info(article):
    nom = article.find("h2")
    adresse = article.find("address")
    telephone = article.find("a", href=lambda href: href and "tel" in href)
    email = article.find("a", href=lambda href: href and "@" in href)
    
    # Vérifier si les balises existent avant d'extraire le texte
    if nom:
        nom = nom.text.strip()
    else:
        nom = ""
    
    if adresse:
        adresse = adresse.text.strip().replace("<br/>", ", ")
    else:
        adresse = ""
    
    if telephone:
        telephone = telephone.text.strip()
    else:
        telephone = ""
    
    if email:
        email = email.text.strip()
    else:
        email = ""
    
    return {"nom": nom, "adresse": adresse, "telephone": telephone, "email": email}


In [45]:
# Fonction pour scraper la page
def scrape_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article")
    article_info_list = [extract_article_info(article) for article in articles]
    return pd.DataFrame(article_info_list)

In [46]:
# Scraper plusieurs pages en gérant la pagination
def scrape_multiple_pages(base_url, num_pages):
    df_list = []
    for page_num in range(1, num_pages + 1):
        url = f"{base_url}?page={page_num}"
        df_list.append(scrape_page(url))
    return pd.concat(df_list, ignore_index=True)

In [47]:
# Utilisation de la fonction pour scraper les données de plusieurs pages
base_url = "https://www.goafricaonline.com/ci/annuaire/distributeur-grossiste?p="
num_pages = 4
df = scrape_multiple_pages(base_url, num_pages)
df

,nom,adresse,telephone,email
0,PROSUMA (SOCIETE IVOIRIENNE DE PROMOTION DE SU...,Rue des Carrossiers - Zone 3Treichville - 01 B...,Tel: (+225) 27 21 25 34 16,
1,DISTRI AKM,"Imm. Pharmacie du Château, 1er étage - Angré C...",Gsm: (+225) 01 01 01 26 12,
2,ABOUSSOUAN SERVICES,Angré ChateauCocody - 05 BP 3303 Abidjan 05Abi...,Gsm: (+225) 05 04 49 73 49,
3,AFRICA DISTRIBUTION,"Avenue 1, face au Palais de la CultureTreichvi...",Tel: (+225) 27 21 35 71 32,
4,AFRICA GLOBAL MARKET,Derrière la Paroisse Saint Bernard - Riviera A...,Tel: (+225) 25 22 01 62 21,
...,...,...,...,...
115,DIMA,01 BP 66 San-Pedro 01San-Pédro - Côte d’Ivoire,Tel: (+225) 27 34 71 52 87,
116,DISTRIBUTION PIECES INDUSTRIELLES,01 BP 794 Abidjan 01San-Pédro - Côte d’Ivoire,Tel: (+225) 27 34 71 96 89,
117,DISTRICOM,"Station essenci, route d'OuméBP 2487Yamoussouk...",Tel: (+225) 27 30 64 61 54,
118,DISTRIMED,-YopougonAbidjan - Côte d’Ivoire,Gsm: (+225) 07 09 87 33 11,


In [48]:
df.to_csv('fournisseurs.csv', index=False)